In [ ]:
# import necessary libraries
import tensorflow as tf
import csv
import numpy as np
import matplotlib.pyplot as plt
import pickle
#%matplotlib inline
from sklearn.manifold import TSNE
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import pandas as pd
from datetime import datetime
import scipy as scp
import math

In [ ]:
# read the resulting factorization with K = 10
[V_res, U_bar_res, U_hat_res] = pickle.load(open('Data/Results/dynMF_10.p', 'rb'))
M, K = V_res.shape
N, _ = U_bar_res.shape
T, _, _ = U_hat_res.shape

In [ ]:
# read corresponding clock time for each time step
tensortime=[]
ll = []
with open('tensor_time.csv') as ff:
    for i, line in enumerate(ff):
        l1 = line.strip("\n")
        ll.append(l1)
        l2 = l1[-19:]
        l2 = datetime.strptime(l2,"%Y-%m-%d %H:%M:%S")
        tensortime.append(l2)

In [ ]:
# correlation between 10 latent dimensions for each time step using U_hat
# the resulting frames will be subsequently used to produce video Video/corr.mp4
for t in range(T):
    print(t)
    temp = []
    correlation = np.zeros((K, K))
    for k1 in range(K):
        for k2 in range(K):
            correlation[k1, k2], _ = scp.stats.pearsonr(U_hat_res[t, :, k1], U_hat_res[t, :, k2])
            if math.isnan(correlation[k1, k2]):
                 print('t = ' + str(t) + '\tk1 = ' + str(k1) + '\tk2 = ' + str(k2))
            else:
                temp.append([k1, k2, correlation[k1, k2]])

    df = pd.DataFrame(data = temp, columns=['latent_dimension1', 'latent_dimension2', 'correlation'])
    df = df.pivot("latent_dimension1", "latent_dimension2", "correlation")
    sns.set(font_scale=1)
    g = sns.heatmap(df, square=True, annot=True, fmt=".1f", vmin=-1.0, vmax=1.0)
    plt.title(tensortime[t])
    #plt.show()
    plt.savefig('./Video/correlation_%04d.png'%t)
    plt.close('all')

In [ ]:
# correlation between 10 latent dimensions using time-independent component U_bar
# the resulting figure is also Figure 4 in the paper
correlation = np.zeros((K, K))
temp = []
for k1 in range(K):
    for k2 in range(K):
        correlation[k1, k2], _ = scp.stats.pearsonr(U_bar_res[:, k1], U_bar_res[:, k2])
        if math.isnan(correlation[k1, k2]):
             print('t = ' + str(t) + '\tk1 = ' + str(k1) + '\tk2 = ' + str(k2))
        else:
            temp.append([k1, k2, correlation[k1, k2]])

df = pd.DataFrame(data = temp, columns=['latent_dimension1', 'latent_dimension2', 'correlation'])
df = df.pivot("latent_dimension1", "latent_dimension2", "correlation")
sns.set(font_scale=1)
g = sns.heatmap(df, square=True, annot=True, fmt=".1f",vmin=-1.0, vmax=1.0)
#plt.show()
plt.savefig('./Figure/correlation.pdf')
plt.close('all')